In [2]:
import os
import pickle
import torch
import numpy as np
from math import ceil
from model_vc import Generator
import librosa
import librosa.display
import matplotlib.pyplot as plt

import math
import glob

baseline = pickle.load(open('/home/ytang363/7100_voiceConversion/results/002_baseline-results.pkl', "rb"))
retrain = pickle.load(open('/home/ytang363/7100_voiceConversion/results/002_results.pkl', "rb"))

In [3]:
# function: https://dsp.stackexchange.com/questions/56391/mel-cepstral-distortion
def log_spec_dB_dist(x, y):
    log_spec_dB_const = 10.0 / math.log(10.0) * math.sqrt(2.0)
    diff = x - y
    return log_spec_dB_const * math.sqrt(np.inner(diff, diff))

cost_function = log_spec_dB_dist

In [4]:
# Load ground truth utterance
rootDir = '/home/ytang363/7100_voiceConversion/VCTK-Corpus-0.92/spmel-16k'
dirName, subdirList, _ = next(os.walk(rootDir))
process_speakers = ['p225', 'p226', 'p227', 'p228', 'p229', 'p230', 'p231', 'p232', 'p233', 'p234', 'p237']
process_uttr = '002'
speakers = []
gt = {}

for speaker in sorted(subdirList):
    if speaker not in process_speakers:
        continue

    print('Processing speaker: %s' % speaker)
    _, _, fileList = next(os.walk(os.path.join(dirName,speaker)))

    indices = [i for i, element in enumerate(fileList) if process_uttr in element]
    print(fileList[indices[0]]) 

    spec = np.load(os.path.join(dirName, speaker, fileList[indices[0]]))

    gt[speaker] = spec

Processing speaker: p225
p225_002_mic1.npy
Processing speaker: p226
p226_002_mic1.npy
Processing speaker: p227
p227_002_mic1.npy
Processing speaker: p228
p228_002_mic1.npy
Processing speaker: p229
p229_002_mic1.npy
Processing speaker: p230
p230_002_mic1.npy
Processing speaker: p231
p231_002_mic1.npy
Processing speaker: p232
p232_002_mic1.npy
Processing speaker: p233
p233_002_mic1.npy
Processing speaker: p234
p234_002_mic1.npy
Processing speaker: p237
p237_002_mic1.npy


In [5]:
# Loop for evaluation
def avg_mcd(gen_mel):
    total_diff = 0
    for i in range(len(process_speakers)):
        current = gen_mel[i][0].split('x')[1]
        test_uttr = gen_mel[i][1] # for p225-001, the shape is (129,80)
        gt_uttr = gt[current]

        min_cost, _ = librosa.sequence.dtw(gt_uttr[:, 1:].T, test_uttr[:, 1:].T, 
                                            metric=cost_function)  
        print(gt_uttr.shape)
        print(test_uttr.shape)
        print(gt_uttr[:, 1:].shape)
        print('----')                                                                            
        diff = np.mean(min_cost) / gt_uttr.shape[0]
        total_diff += diff
    avg_MCD = total_diff / len(process_speakers)
    return avg_MCD       

In [6]:
avg_mcd(baseline)

(247, 80)
(247, 80)
(247, 79)
----
(270, 80)
(247, 80)
(270, 79)
----
(267, 80)
(247, 80)
(267, 79)
----
(308, 80)
(247, 80)
(308, 79)
----
(243, 80)
(247, 80)
(243, 79)
----
(279, 80)
(247, 80)
(279, 79)
----
(203, 80)
(247, 80)
(203, 79)
----
(193, 80)
(247, 80)
(193, 79)
----
(283, 80)
(247, 80)
(283, 79)
----
(249, 80)
(247, 80)
(249, 79)
----
(264, 80)
(247, 80)
(264, 79)
----


4.640020417134696

In [7]:
avg_mcd(retrain)

(247, 80)
(247, 80)
(247, 79)
----
(270, 80)
(247, 80)
(270, 79)
----
(267, 80)
(247, 80)
(267, 79)
----
(308, 80)
(247, 80)
(308, 79)
----
(243, 80)
(247, 80)
(243, 79)
----
(279, 80)
(247, 80)
(279, 79)
----
(203, 80)
(247, 80)
(203, 79)
----
(193, 80)
(247, 80)
(193, 79)
----
(283, 80)
(247, 80)
(283, 79)
----
(249, 80)
(247, 80)
(249, 79)
----
(264, 80)
(247, 80)
(264, 79)
----


4.486204718444406